## Importing

In [ ]:
%%capture
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import gc #garbage collector for gpu memory 
from tqdm import tqdm
import json
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig, AutoModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Loading Data

In [ ]:
politifact_data = json.load(open("gdrive/MyDrive/BT4222/Data/politifact_combined.json", "r"))
gossipcop_data = json.load(open("gdrive/MyDrive/BT4222/Data/gossipcop_combined.json", "r"))

In [ ]:
politifact_df = pd.DataFrame(politifact_data)
gossipcop_df = pd.DataFrame(gossipcop_data)

In [ ]:
politifact_df['target'] = politifact_df['label'].apply(lambda x: 1 if x=='real' else 0)
gossipcop_df['target'] = gossipcop_df['label'].apply(lambda x: 1 if x=='real' else 0)

In [ ]:
politifact_df['parsed_month'] = politifact_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%m") if not pd.isna(x) else '0')
gossipcop_df['parsed_month'] = gossipcop_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%m") if not pd.isna(x) else '0')

In [ ]:
politifact_df['parsed_hour'] = politifact_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%H") if not pd.isna(x) else '0')
gossipcop_df['parsed_hour'] = gossipcop_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%H") if not pd.isna(x) else '0')

In [ ]:
politifact_df['publisher'] = politifact_df['publisher'].fillna('None')
gossipcop_df['publisher'] = gossipcop_df['publisher'].fillna('None')

## Choose type of article to run

In [ ]:
# Choose gossipcop or politifact
article = "politifact"

In [ ]:
if article == "politifact":
  X_extra = politifact_df[['parsed_hour','parsed_month', 'publisher']]
else:
  X_extra = gossipcop_df[['parsed_hour','parsed_month', 'publisher']]

In [ ]:
X_extra['parsed_hour'] = X_extra['parsed_hour'].astype('str')
X_extra['parsed_month'] = X_extra['parsed_month'].astype('str')
X_extra['publisher'] = X_extra['publisher'].astype('str')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

## Tokenizing - TF-idf

We don't use CV as it performs much poorly

In [ ]:
def tokenize(df):
    # Get tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    # tokenize text
    print("Title Tokenizing")
    title_tokenized_df = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:98] + ['[SEP]'], tqdm(df['title'])))
    # Get token index
    title_indexed_tokens = list(map(tokenizer.convert_tokens_to_ids, title_tokenized_df))
    
    # Pad tokens
    totalpadlength = 100
    title_index_padded = np.array([xi+[0]*(totalpadlength-len(xi)) for xi in title_indexed_tokens])

    # Mask
    title_mask_variable = [[float(i>0) for i in ii] for ii in title_index_padded]


    print("Article Tokenizing")
    article_tokenized_df = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], tqdm(df['text'])))    
    article_indexed_tokens = list(map(tokenizer.convert_tokens_to_ids, article_tokenized_df))
  
    # Pad tokens
    totalpadlength = 512
    article_index_padded = np.array([xi+[0]*(totalpadlength-len(xi)) for xi in article_indexed_tokens])

    # Mask
    article_mask_variable = [[float(i>0) for i in ii] for ii in article_index_padded]
  
    # Article text
    tweets_text = df.tweets_text.values

    # Target Variable
    target_variable = df['target'].values

    return title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, tweets_text, target_variable

def format_tensors(article_data, article_mask, title_data, title_mask, extra_features, tweets_text, labels, batch_size):
    
    X_article = torch.from_numpy(article_data)
    X_article = X_article.long()
    article_mask = torch.tensor(article_mask)

    X_title = torch.from_numpy(title_data)
    X_title = X_title.long()
    title_mask = torch.tensor(title_mask)

    extra_features = torch.from_numpy(extra_features)
    extra_features = extra_features.long()

    tweets_text = torch.from_numpy(tweets_text)
    tweets_text = tweets_text.long()    

    y = torch.from_numpy(labels)
    y = y.long()

    tensordata = data_utils.TensorDataset(X_article, article_mask, X_title, title_mask, extra_features,tweets_text,  y)
    loader = data_utils.DataLoader(tensordata, batch_size=batch_size, shuffle=False)
    
    return loader

def train_validation_test(title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, tweets_text, target_variable, BATCH_SIZE = 8):

    # Train test split for train set
    X_train_title, X_rest_title, y_train, y_rest = train_test_split(title_index_padded, target_variable, test_size=0.3, random_state=42)
    train_masks_title, rest_masks_title, _, _ = train_test_split(title_mask_variable, title_index_padded, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_title, X_test_title, y_val, y_test = train_test_split(X_rest_title, y_rest, test_size=0.5, random_state=42)
    val_masks_title, test_masks_title, _, _ = train_test_split(rest_masks_title, X_rest_title, test_size=0.5, random_state=42)

    # Train test split for train set
    X_train_article, X_rest_article, y_train, y_rest = train_test_split(article_index_padded, target_variable, test_size=0.3, random_state=42)
    train_masks_article, rest_masks_article, _, _ = train_test_split(article_mask_variable, article_index_padded, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_article, X_test_article, y_val, y_test = train_test_split(X_rest_article, y_rest, test_size=0.5, random_state=42)
    val_masks_article, test_masks_article, _, _ = train_test_split(rest_masks_article, X_rest_article, test_size=0.5, random_state=42)


    # Tweets #
    # Train test split for train set
    X_train_tweet, X_rest_tweet, _, _ = train_test_split(tweets_text, target_variable, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_tweet, X_test_tweet, _, _ = train_test_split(X_rest_tweet, y_rest, test_size=0.5, random_state=42)

    vect = TfidfVectorizer(max_features=10000)
    X_train_tweet_dtm = vect.fit_transform(X_train_tweet).toarray()
    X_test_tweet_dtm = vect.transform(X_test_tweet).toarray()
    X_val_tweet_dtm = vect.transform(X_val_tweet).toarray()

    # Extra Features #
    # Train test split for train set
    X_train_extra, X_rest_extra, _, _ = train_test_split(X_extra, target_variable, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_extra, X_test_extra, _, _ = train_test_split(X_rest_extra, y_rest, test_size=0.5, random_state=42)

    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc.fit(X_train_extra)
    X_train_extra_enc = enc.transform(X_train_extra).toarray()
    X_test_extra_enc = enc.transform(X_test_extra).toarray()
    X_val_extra_enc = enc.transform(X_val_extra).toarray()


    trainloader = format_tensors(X_train_article, train_masks_article, X_train_title, train_masks_title, X_train_extra_enc, X_train_tweet_dtm, y_train, BATCH_SIZE)
    validationloader = format_tensors(X_val_article, val_masks_article, X_val_title, val_masks_title, X_val_extra_enc, X_val_tweet_dtm, y_val, BATCH_SIZE)
    testloader = format_tensors(X_test_article, test_masks_article, X_test_title, test_masks_title, X_test_extra_enc,X_test_tweet_dtm,  y_test, BATCH_SIZE)

    return trainloader, validationloader, testloader

In [ ]:
title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, tweets_text, target_variable = tokenize(politifact_df) if article == "politifact" else tokenize(gossipcop_df)
trainloader, validationloader, testloader = train_validation_test(title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, tweets_text, target_variable)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Title Tokenizing


100%|██████████| 954/954 [00:00<00:00, 1199.60it/s]


Article Tokenizing


100%|██████████| 954/954 [00:46<00:00, 20.41it/s]


# Model Definition

In [ ]:
class TwoBertTweetsMore(torch.nn.Module):
    """
    This takes a transformer backbone and puts a slightly-modified classification head on top.
    
    """

    def __init__(self,article_model_name, title_model_name, extra_dims, num_labels):
        # num_extra_dims corresponds to the number of extra dimensions of numerical/categorical data

        super().__init__()

        # self.config = AutoConfig.from_pretrained(model_name)
        self.article_transformer = AutoModel.from_pretrained(title_model_name)
        self.title_transformer = AutoModel.from_pretrained(title_model_name) #title transformer
        
        for param in self.article_transformer.parameters():
          param.requires_grad = False        

        for param in self.title_transformer.parameters():
          param.requires_grad = False
        
        num_hidden_size_1 = self.article_transformer.config.hidden_size
        num_hidden_size_2 = self.title_transformer.config.hidden_size # May be different depending on which model you use. Common sizes are 768 and 1024. Look in the config.json file 
        
        input_dim = +num_hidden_size_1 + num_hidden_size_2 + extra_dims
        
        self.fc1 = torch.nn.Linear(input_dim, 100)
        self.fc2 = torch.nn.Linear(100, 100)
        #self.classifier = torch.nn.Linear(input_dim, 2)
        self.classifier = nn.Linear(100,2)
        self.dropout = torch.nn.Dropout(0.25)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, article_input_ids, article_attention_mask, title_input_ids, title_attention_mask, extra_features, tweet_text):
        """
        extra_data should be of shape [batch_size, dim] 
        where dim is the number of additional numerical/categorical dimensions
        """

        article_hidden_states = self.title_transformer(input_ids=article_input_ids, attention_mask=article_attention_mask)

        article_cls_embeds = article_hidden_states.last_hidden_state[:, 0, :] # [batch size, hidden size]
        
        title_hidden_states = self.title_transformer(input_ids=title_input_ids, attention_mask=title_attention_mask) # [batch size, sequence length, hidden size]

        title_cls_embeds = title_hidden_states.last_hidden_state[:, 0, :] # [batch size, hidden size]

        concat = torch.cat((article_cls_embeds, title_cls_embeds, extra_features,tweet_text), dim=-1) # [batch size, hidden size+num extra dims]

        # return output
        x = self.fc1(concat)
        x = self.relu1(x)
        x = self.dropout(x) 
       
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.relu2(x) 
        
        output = self.classifier(x) # [batch size, num labels]

        return output

# Model Creation

In [ ]:
def train_model(epochs, model, learning_rate, start_from_epoch=1):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache() #memory
    gc.collect() #memory
    NUM_EPOCHS = epochs
    loss_function = nn.CrossEntropyLoss()
    losses = []
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_acc, best_roc_auc = 0,0
    for epoch in range(start_from_epoch, NUM_EPOCHS+1):
        model.train()

        # For epoch metrics
        epoch_loss = 0.0
        preds, truth, pred_proba = [],[],[]
        iteration = 0
        for i, batch in enumerate(tqdm(trainloader)):
            iteration += 1
            article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text, labels = tuple(t.to(device) for t in batch)
            optimizer.zero_grad()
            outputs = model(article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text)
            loss = loss_function(outputs, labels)
            epoch_loss += float(loss.item())
            yhat = outputs
            loss.backward()
            optimizer.step()

            # Metrics for batch
            prediction_proba = torch.sigmoid(yhat[:,1]).cpu().data.numpy()
            prediction = (prediction_proba > 0.5).astype(int)
            baseline = labels.long().cpu().data.numpy().astype(int)
            preds.extend(prediction)
            pred_proba.extend(prediction_proba)
            truth.extend(baseline)

            del article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text, labels #memory
            torch.cuda.empty_cache() #memory
            gc.collect() #memory
        

        # Calculate train and validation metrics and log them
        with torch.set_grad_enabled(False):
            metrics = {}
            # Training
            avg_accuracy, avg_roc_auc, avg_f1, avg_loss = accuracy_score(truth, preds), roc_auc_score(truth, pred_proba), f1_score(truth, preds), epoch_loss/float(iteration)
            print(f'Epoch {epoch}:\nTraining Accuracy: 'f'{avg_accuracy:.4f}%')
            print(f'Training ROC AUC: 'f'{avg_roc_auc:.4f}%')
            print(f'Training F1: 'f'{avg_f1:.4f}%')
            print(f'Training loss: 'f'{avg_loss}%\n')
            metrics['train'] = {
                'accuracy':avg_accuracy,
                'roc_auc':avg_roc_auc,
                'f1':avg_f1,
                'loss':avg_loss
            }

            # Validation
            model.eval()
            epoch_loss = 0.0
            preds, truth, pred_proba = [],[],[]
            iteration = 0
            with torch.no_grad():
                for i, batch in enumerate(tqdm(validationloader)):
                    iteration += 1
                    article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text, labels = tuple(t.to(device) for t in batch)
                    outputs = model(article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text)
                    
                    loss = loss_function(outputs, labels)
                    epoch_loss += float(loss.item())
                    yhat = outputs

                    prediction_proba = torch.sigmoid(yhat[:,1]).cpu().data.numpy()
                    prediction = (prediction_proba > 0.5).astype(int)
                    baseline = labels.long().cpu().data.numpy().astype(int)
                    preds.extend(prediction)
                    pred_proba.extend(prediction_proba)
                    truth.extend(baseline)

                    del article_token_ids, article_masks, title_token_ids, title_masks, extra_features, tweet_text, labels #memory
                    torch.cuda.empty_cache() #memory
                    gc.collect() #memory

            avg_accuracy, avg_roc_auc, avg_f1, avg_loss = accuracy_score(truth, preds), roc_auc_score(truth, pred_proba), f1_score(truth, preds), epoch_loss/float(iteration)
            print(f'Validation Accuracy: 'f'{avg_accuracy:.4f}%')
            print(f'Validation ROC AUC: 'f'{avg_roc_auc:.4f}%')
            print(f'Validation F1: 'f'{avg_f1:.4f}%')
            print(f'Validation loss: 'f'{avg_loss}%\n')
            metrics['validation'] = {
                'accuracy':avg_accuracy,
                'roc_auc':avg_roc_auc,
                'f1':avg_f1,
                'loss':avg_loss
            }


In [ ]:
article_model_name = f"gdrive/MyDrive/BT4222/Code/machine_learning/xp/{article}/distilbert_fulltext/model_epoch2"
title_model_name = f"distilbert-base-uncased"
# Path to the best title distilbert model we trained
# title_model_name = f"gdrive/MyDrive/BT4222/Code/machine_learning/xp/{article}/distilbert_title/model_epoch2" 
num_labels = 2
num_extra_dims = total_dims

model = TwoBertTweetsMore(article_model_name, title_model_name, num_extra_dims, num_labels)

Some weights of the model checkpoint at gdrive/MyDrive/BT4222/Code/machine_learning/xp/politifact/distilbert_title/model_epoch2 were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at gdrive/MyDrive/BT4222/Code/machine_learning/xp/politifact/distilbert_title/model_epoch2 were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
- T

In [ ]:
train_model(20, model, 3e-5, start_from_epoch=1)

100%|██████████| 84/84 [00:44<00:00,  1.90it/s]


Epoch 1:
Training Accuracy: 0.8576%
Training ROC AUC: 0.9381%
Training F1: 0.8697%
Training loss: 0.5915658658459073%



100%|██████████| 18/18 [00:08<00:00,  2.07it/s]


Validation Accuracy: 0.9021%
Validation ROC AUC: 0.9523%
Validation F1: 0.9167%
Validation loss: 0.4682723631461461%



100%|██████████| 84/84 [00:43<00:00,  1.91it/s]


...

Validation Accuracy: 0.8881%
Validation ROC AUC: 0.9531%
Validation F1: 0.9048%
Validation loss: 0.30006742172150147%



100%|██████████| 84/84 [00:42<00:00,  1.96it/s]


Epoch 20:
Training Accuracy: 0.9385%
Training ROC AUC: 0.9826%
Training F1: 0.9476%
Training loss: 0.15591716496939106%



100%|██████████| 18/18 [00:08<00:00,  2.05it/s]

Validation Accuracy: 0.8811%
Validation ROC AUC: 0.9535%
Validation F1: 0.8982%
Validation loss: 0.29495972896822625%

